# Spectra acquisiton
Control both the stepping motor and the camera from this notebook.<br>
Make a loop to measure a wide-range spectrum automatically.

NOTE: only one connection to COM port is possible at a time. Shut down other connections, either in python or dedicated software.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import time, os, logging
from FLI import FLI
import numpy as np
import xarray as xr
from os.path import join

In [3]:
%run thr640.py
%run imaging_tool.py

In [4]:
logger = logger
logger.setLevel(50) # 10 - DEBUG 50 - CRITICAL 20 - INFO
camera = FLI()
step_motor = THR640() # Can only have one instance.

In [5]:
basepath = r'\\10.249.254.51\Public\Fujii\data\hongu\2020.10.05'
os.path.exists(basepath)

True

In [6]:
join(basepath,'a.nc')

'\\\\10.249.254.51\\Public\\Fujii\\data\\hongu\\2020.10.05\\a.nc'

In [7]:
exposure = 1.5 * 1e2 # ms
vbin = 1
attributes = {
    "temperature": camera.getTemperature(),
    "device_status": camera.getDeviceStatus(),
    "exposure": exposure,
    "frame_type": "light",
    "motor_coordinate":"",
}

camera.setExposureTime(exposure)
camera.setVBin(vbin)
camera.setImageArea(10,0,2058,512//vbin)
time.sleep(.5)

R = [410000 + i*5000 for i in range(2)]
for r in log_progress(R):
    step_motor.goto(r)
    step_motor.waitUntilReady()
    attributes['motor_coordinate'] = r
    
    camera.exposeFrame()
    time.sleep(.1)
    # load image
    image_data = camera.grabFrame(out=np.empty((512//vbin,2048), np.uint16))
    # convert to xarray
    data = xr.DataArray(
        image_data, dims=["y", "x"], coords={"image_counter": r}, attrs=attributes
    )
    # save data as NetCDF file
    filepath = join(basepath,f'H_{r}.nc')
    print(filepath)
    data.to_netcdf(filepath)
print('Done.')

\\10.249.254.51\Public\Fujii\data\hongu\2020.10.05\H_410000.nc
\\10.249.254.51\Public\Fujii\data\hongu\2020.10.05\H_415000.nc
Done.


In [8]:
[400000 + i*9000 for i in range(15)]

[400000,
 409000,
 418000,
 427000,
 436000,
 445000,
 454000,
 463000,
 472000,
 481000,
 490000,
 499000,
 508000,
 517000,
 526000]